# 🧪 Review Reviewer — Interactive Demo Notebook

This notebook demonstrates our hackathon project:  
**Automatically flagging irrelevant or low-quality Google reviews (text + image) to improve trust in local business platforms.**

We show two workflows:
1. **Text-only** review classification.  
2. **Image-augmented** classification (generate a caption for an attached image, merge with review text, then classify).

The outputs show whether a review is:
- `advertisement`
- `irrelevant`
- `rant_no_visit`
- `image_irrelevant`
- `image_advertisement`
- `ok`

…and whether it should be considered **relevant** or not.

## 1. Setup & Environment

This demo requires:
- Python 3.9+  
- Dependencies: `groq`, `transformers`, `torch`, `python-dotenv`, `pandas`, `Pillow`
- Create a .env file in the repo root with your GROP API key

> If needed, install with:
```bash
pip install -U groq transformers torch python-dotenv pandas pillow

## 2. Quick Test — Text-only Review

We first classify a simple review string without any image attached.  
Expected outcome: detect `rant_no_visit`, `irrelevant`, etc. based purely on text.

In [1]:
from review_classifier import classify_review, get_groq_client

# Reuse the Groq client for efficiency
client = get_groq_client()

# Example review payload
review = {
    "category": "Pizza restaurant",
    "rating": 1,
    "text": "Never been here but I heard it's terrible.",
    "place_name": "Slicetown",
}

# Run classification
result = classify_review(
    category=review["category"],
    rating=review["rating"],
    text=review["text"],
    place_name=review["place_name"],
    client=client
)

print(result)

{'relevant': False, 'violation': 'rant_no_visit', 'classification': 'rant_no_visit', 'confidence': 1.0, 'reasoning': 'reviewer explicitly stated they did not visit', 'indicators': []}


## 3. Quick Test — Review with Image

If a review has an image:
1. Generate a caption with BLIP.  
2. Append caption to the review text as:  
   `"With a picture of <caption>"`.  
3. Pass into classifier with new rules (`image_irrelevant` or `image_advertisement`).  

This demonstrates **multimodal classification**.

In [2]:
from image_augmented_classifier import classify_review_with_image
from review_classifier import get_groq_client

# Reuse the Groq client for efficiency
client = get_groq_client()

# Example review with a local image file
review = {
    "category": "Pizza restaurant",
    "rating": 2,
    "text": "The dough was soggy but service was friendly.",
    "place_name": "Mario's Slice",
    "image_path": "../image/img_1.jpg",  # <-- ensure this exists
}

result = classify_review_with_image(
    category=review["category"],
    rating=review["rating"],
    text=review["text"],
    place_name=review["place_name"],
    image_path=review["image_path"],
    client=client,
)

print(result)
# Example fields:
# {
#   "relevant": true/false,
#   "violation": "ok"|"image_irrelevant"|"image_advertisement"|...,
#   "classification": "...",
#   "confidence": 0.8,
#   "reasoning": "...",
#   "indicators": [...],
#   "_image_caption": "a slice of pizza on a plate"  # for auditing
# }

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

{'relevant': False, 'violation': 'image_irrelevant', 'classification': 'image_irrelevant', 'confidence': 0.8, 'reasoning': 'image description contradicts the place category', 'indicators': ['irrelevant_picture'], '_image_caption': "a woman ' s hand with a ring on it"}
